In [ ]:
from pyspark.sql.types import *

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, StringType

# Create a Spark session
spark = SparkSession.builder.appName("MovieLensAnalysis").getOrCreate()

# Define the schema to ensure each columns has suitable type
schema = StructType().add('movieId', IntegerType()).add('title', StringType()).add('genres', StringType())

# Load the MovieLens dataset
movie_df = spark.read.csv('movies.csv', schema=schema, header=True)

# Show the DataFrame
movie_df.show()


+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [3]:
# transformation on dataset
from pyspark.sql.functions import split ,explode
movie_df = movie_df.withColumn('genres',explode(split('genres','[|]')))
movie_df.show()

+-------+--------------------+---------+
|movieId|               title|   genres|
+-------+--------------------+---------+
|      1|    Toy Story (1995)|Adventure|
|      1|    Toy Story (1995)|Animation|
|      1|    Toy Story (1995)| Children|
|      1|    Toy Story (1995)|   Comedy|
|      1|    Toy Story (1995)|  Fantasy|
|      2|      Jumanji (1995)|Adventure|
|      2|      Jumanji (1995)| Children|
|      2|      Jumanji (1995)|  Fantasy|
|      3|Grumpier Old Men ...|   Comedy|
|      3|Grumpier Old Men ...|  Romance|
|      4|Waiting to Exhale...|   Comedy|
|      4|Waiting to Exhale...|    Drama|
|      4|Waiting to Exhale...|  Romance|
|      5|Father of the Bri...|   Comedy|
|      6|         Heat (1995)|   Action|
|      6|         Heat (1995)|    Crime|
|      6|         Heat (1995)| Thriller|
|      7|      Sabrina (1995)|   Comedy|
|      7|      Sabrina (1995)|  Romance|
|      8| Tom and Huck (1995)|Adventure|
+-------+--------------------+---------+
only showing top

#### 1- withColumn: is used when need to apply condition on the column ,if the column exist will apply the condtion ,
####  if not it will create the new column and apply the condition


#### 2 - explode: used to transform any array or map column into multiple rows with  one row (means when film contain more than one genre will be duplicate for each genre

#### --> we split genre by [|]

In [5]:
# count number of movies accroding to genre
movie_df.groupBy('genres').count().show()

+------------------+-----+
|            genres|count|
+------------------+-----+
|             Crime| 1199|
|           Romance| 1596|
|          Thriller| 1894|
|         Adventure| 1263|
|             Drama| 4361|
|               War|  382|
|       Documentary|  440|
|           Fantasy|  779|
|           Mystery|  573|
|           Musical|  334|
|         Animation|  611|
|         Film-Noir|   87|
|(no genres listed)|   34|
|              IMAX|  158|
|            Horror|  978|
|           Western|  167|
|            Comedy| 3756|
|          Children|  664|
|            Action| 1828|
|            Sci-Fi|  980|
+------------------+-----+



In [6]:
movie_df.printSchema()
movie_df.show()


root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+--------------------+---------+
|movieId|               title|   genres|
+-------+--------------------+---------+
|      1|    Toy Story (1995)|Adventure|
|      1|    Toy Story (1995)|Animation|
|      1|    Toy Story (1995)| Children|
|      1|    Toy Story (1995)|   Comedy|
|      1|    Toy Story (1995)|  Fantasy|
|      2|      Jumanji (1995)|Adventure|
|      2|      Jumanji (1995)| Children|
|      2|      Jumanji (1995)|  Fantasy|
|      3|Grumpier Old Men ...|   Comedy|
|      3|Grumpier Old Men ...|  Romance|
|      4|Waiting to Exhale...|   Comedy|
|      4|Waiting to Exhale...|    Drama|
|      4|Waiting to Exhale...|  Romance|
|      5|Father of the Bri...|   Comedy|
|      6|         Heat (1995)|   Action|
|      6|         Heat (1995)|    Crime|
|      6|         Heat (1995)| Thriller|
|      7|      Sabrina (1995)|   Comedy|
|      7|      Sab

In [7]:
# count movies accroding genres
movie_df.groupBy('genres').count().show()

+------------------+-----+
|            genres|count|
+------------------+-----+
|             Crime| 1199|
|           Romance| 1596|
|          Thriller| 1894|
|         Adventure| 1263|
|             Drama| 4361|
|               War|  382|
|       Documentary|  440|
|           Fantasy|  779|
|           Mystery|  573|
|           Musical|  334|
|         Animation|  611|
|         Film-Noir|   87|
|(no genres listed)|   34|
|              IMAX|  158|
|            Horror|  978|
|           Western|  167|
|            Comedy| 3756|
|          Children|  664|
|            Action| 1828|
|            Sci-Fi|  980|
+------------------+-----+



In [8]:
movie_df.filter(movie_df.genres =="(no genres listed)").show()
# check if there movies with no specified genre

+-------+--------------------+------------------+
|movieId|               title|            genres|
+-------+--------------------+------------------+
| 114335|   La cravate (1957)|(no genres listed)|
| 122888|      Ben-hur (2016)|(no genres listed)|
| 122896|Pirates of the Ca...|(no genres listed)|
| 129250|   Superfast! (2015)|(no genres listed)|
| 132084| Let It Be Me (1995)|(no genres listed)|
| 134861|Trevor Noah: Afri...|(no genres listed)|
| 141131|    Guardians (2016)|(no genres listed)|
| 141866|   Green Room (2015)|(no genres listed)|
| 142456|The Brand New Tes...|(no genres listed)|
| 143410|          Hyena Road|(no genres listed)|
| 147250|The Adventures of...|(no genres listed)|
| 149330|A Cosmic Christma...|(no genres listed)|
| 152037|  Grease Live (2016)|(no genres listed)|
| 155589|Noin 7 veljestä (...|(no genres listed)|
| 156605|            Paterson|(no genres listed)|
| 159161|Ali Wong: Baby Co...|(no genres listed)|
| 159779|A Midsummer Night...|(no genres listed)|


In [9]:
# replace missng genere with unknown
from pyspark.sql.functions import when
movie_df = movie_df.withColumn('genres',when(movie_df.genres =="(no genres listed)","Unknown").otherwise(movie_df.genres))
movie_df.show()

+-------+--------------------+---------+
|movieId|               title|   genres|
+-------+--------------------+---------+
|      1|    Toy Story (1995)|Adventure|
|      1|    Toy Story (1995)|Animation|
|      1|    Toy Story (1995)| Children|
|      1|    Toy Story (1995)|   Comedy|
|      1|    Toy Story (1995)|  Fantasy|
|      2|      Jumanji (1995)|Adventure|
|      2|      Jumanji (1995)| Children|
|      2|      Jumanji (1995)|  Fantasy|
|      3|Grumpier Old Men ...|   Comedy|
|      3|Grumpier Old Men ...|  Romance|
|      4|Waiting to Exhale...|   Comedy|
|      4|Waiting to Exhale...|    Drama|
|      4|Waiting to Exhale...|  Romance|
|      5|Father of the Bri...|   Comedy|
|      6|         Heat (1995)|   Action|
|      6|         Heat (1995)|    Crime|
|      6|         Heat (1995)| Thriller|
|      7|      Sabrina (1995)|   Comedy|
|      7|      Sabrina (1995)|  Romance|
|      8| Tom and Huck (1995)|Adventure|
+-------+--------------------+---------+
only showing top

In [10]:
movie_df.filter(movie_df.genres =="(no genres listed)").show()
# check if there movies with no specified genre

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
+-------+-----+------+



## Rating Dataset

In [11]:
rating_df = spark.read.csv('ratings.csv',header = True)
rating_df.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



### merging the two dataset

In [12]:
df = movie_df.join(rating_df, movie_df.movieId ==rating_df.movieId).drop(rating_df.movieId)
df.show()

+-------+--------------------+---------+------+------+---------+
|movieId|               title|   genres|userId|rating|timestamp|
+-------+--------------------+---------+------+------+---------+
|      1|    Toy Story (1995)|  Fantasy|     1|   4.0|964982703|
|      1|    Toy Story (1995)|   Comedy|     1|   4.0|964982703|
|      1|    Toy Story (1995)| Children|     1|   4.0|964982703|
|      1|    Toy Story (1995)|Animation|     1|   4.0|964982703|
|      1|    Toy Story (1995)|Adventure|     1|   4.0|964982703|
|      3|Grumpier Old Men ...|  Romance|     1|   4.0|964981247|
|      3|Grumpier Old Men ...|   Comedy|     1|   4.0|964981247|
|      6|         Heat (1995)| Thriller|     1|   4.0|964982224|
|      6|         Heat (1995)|    Crime|     1|   4.0|964982224|
|      6|         Heat (1995)|   Action|     1|   4.0|964982224|
|     47|Seven (a.k.a. Se7...| Thriller|     1|   5.0|964983815|
|     47|Seven (a.k.a. Se7...|  Mystery|     1|   5.0|964983815|
|     50|Usual Suspects, 

#### the issue in the above result , the same movie  has more than one genre

In [13]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number
filter_df = df.withColumn('rank',row_number().over(Window.partitionBy('userId', 'title').orderBy(df['rating'].desc())))
filter_df.show()

+-------+--------------------+---------+------+------+---------+----+
|movieId|               title|   genres|userId|rating|timestamp|rank|
+-------+--------------------+---------+------+------+---------+----+
|   1127|   Abyss, The (1989)| Thriller|     1|   4.0|964982513|   1|
|   1127|   Abyss, The (1989)|   Sci-Fi|     1|   4.0|964982513|   2|
|   1127|   Abyss, The (1989)|Adventure|     1|   4.0|964982513|   3|
|   1127|   Abyss, The (1989)|   Action|     1|   4.0|964982513|   4|
|    940|Adventures of Rob...|  Romance|     1|   5.0|964982176|   1|
|    940|Adventures of Rob...|Adventure|     1|   5.0|964982176|   2|
|    940|Adventures of Rob...|   Action|     1|   5.0|964982176|   3|
|    101|Bottle Rocket (1996)|  Romance|     1|   5.0|964980868|   1|
|    101|Bottle Rocket (1996)|    Crime|     1|   5.0|964980868|   2|
|    101|Bottle Rocket (1996)|   Comedy|     1|   5.0|964980868|   3|
|    101|Bottle Rocket (1996)|Adventure|     1|   5.0|964980868|   4|
|   2291|Edward Scis

In [14]:
# get the movies based on rank
filter_df = filter_df.filter(filter_df.rank == 1)
filter_df.show()

+-------+--------------------+--------+------+------+---------+----+
|movieId|               title|  genres|userId|rating|timestamp|rank|
+-------+--------------------+--------+------+------+---------+----+
|   1127|   Abyss, The (1989)|Thriller|     1|   4.0|964982513|   1|
|    940|Adventures of Rob...| Romance|     1|   5.0|964982176|   1|
|    101|Bottle Rocket (1996)| Romance|     1|   5.0|964980868|   1|
|   2291|Edward Scissorhan...| Romance|     1|   5.0|964983664|   1|
|   3243|   Encino Man (1992)|  Comedy|     1|   3.0|964981093|   1|
|    356| Forrest Gump (1994)|     War|     1|   4.0|964980962|   1|
|    943|Ghost and Mrs. Mu...| Romance|     1|   4.0|964983614|   1|
|   3578|    Gladiator (2000)|   Drama|     1|   5.0|964980668|   1|
|   2338|I Still Know What...|Thriller|     1|   2.0|964983546|   1|
|   2000|Lethal Weapon (1987)|   Drama|     1|   4.0|964982211|   1|
|   2991|Live and Let Die ...|Thriller|     1|   5.0|964982271|   1|
|   2116|Lord of the Rings...| Fan

### Find highly rated movie 

In [15]:
from pyspark.sql.functions import max
most_highly_rated_movie = filter_df.groupBy('title').agg(max('rating').alias('max_rating'))
most_highly_rated_movie.show()

+--------------------+----------+
|               title|max_rating|
+--------------------+----------+
|"11'09""01 - Sept...|       4.0|
|          '71 (2014)|       4.0|
|'Hellboy': The Se...|       4.0|
|'Round Midnight (...|       3.5|
| 'Salem's Lot (2004)|       5.0|
|'Til There Was Yo...|       5.0|
|'Tis the Season f...|       1.5|
|  'burbs, The (1989)|       5.0|
|'night Mother (1986)|       3.0|
|(500) Days of Sum...|       5.0|
|*batteries not in...|       4.0|
|...All the Marble...|       2.0|
|...And Justice fo...|       4.0|
|00 Schneider - Ja...|       4.5|
|   1-900 (06) (1994)|       4.0|
|           10 (1979)|       4.5|
|10 Cent Pistol (2...|       2.0|
|10 Cloverfield La...|       5.0|
|10 Items or Less ...|       3.5|
|10 Things I Hate ...|       5.0|
+--------------------+----------+
only showing top 20 rows



### worst movie rating

In [16]:
worst_rated_movies = df.withColumn('rank',row_number().over(Window.orderBy(df['rating'])))
worst_rated_movie = worst_rated_movies.filter(worst_rated_movies['rank']==1)
worst_rated_movie.show()

+-------+--------------------+------+------+------+----------+----+
|movieId|               title|genres|userId|rating| timestamp|rank|
+-------+--------------------+------+------+------+----------+----+
|     31|Dangerous Minds (...| Drama|     3|   0.5|1306463578|   1|
+-------+--------------------+------+------+------+----------+----+



In [17]:
worst_rated_movies.show()

+-------+--------------------+---------+------+------+----------+----+
|movieId|               title|   genres|userId|rating| timestamp|rank|
+-------+--------------------+---------+------+------+----------+----+
|   1093|   Doors, The (1991)|    Drama|     3|   0.5|1306463627|   1|
|   2090|Rescuers, The (1977)|Adventure|     3|   0.5|1306464261|   2|
|   1124|On Golden Pond (1...|    Drama|     3|   0.5|1306464216|   3|
|   1263|Deer Hunter, The ...|      War|     3|   0.5|1306463569|   4|
|   2090|Rescuers, The (1977)|Animation|     3|   0.5|1306464261|   5|
|   1263|Deer Hunter, The ...|    Drama|     3|   0.5|1306463569|   6|
|     31|Dangerous Minds (...|    Drama|     3|   0.5|1306463578|   7|
|   1272|       Patton (1970)|      War|     3|   0.5|1306463624|   8|
|    527|Schindler's List ...|    Drama|     3|   0.5|1306464275|   9|
|   1272|       Patton (1970)|    Drama|     3|   0.5|1306463624|  10|
|    647|Courage Under Fir...|    Drama|     3|   0.5|1306463619|  11|
|   13